In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

# Any results you write to the current directory are saved as output.

In [ ]:
! pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ../input/nvidiaapex/repository/NVIDIA-apex-39e153a

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import datetime
import pkg_resources
import seaborn as sns
import time
import scipy.stats as stats
import gc
import re
import operator 
import sys
from sklearn import metrics
from sklearn import model_selection
import matplotlib.pyplot as plt
#from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import KFold
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
from nltk.stem import PorterStemmer
from sklearn.metrics import roc_auc_score
%load_ext autoreload
%autoreload 2
%matplotlib inline
from tqdm import tqdm, tqdm_notebook
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
from apex import amp
from joblib import Parallel, delayed
import shutil

In [ ]:

device=torch.device('cuda')

In [ ]:
MAX_SEQUENCE_LENGTH = 200
SEED = 1234
num_epoch=1
lr=2e-5
batch_size=32
accumulation_steps=2

Data_dir="../input/jigsaw-unintended-bias-in-toxicity-classification"
Input_dir = "../input"
WORK_DIR = "../working/"
#sequences_path="../input/pickled-bert-sequences/bert_sequences.pkl"
#predict_weights_file = "../input/vesla-trained-bert/bert_classifier.bin"
trained_bert_path = None
os.listdir(WORK_DIR)
partial_train = 0.5 
TOXICITY_COLUMN = 'target'

In [ ]:

from pytorch_pretrained_bert import convert_tf_checkpoint_to_pytorch
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification,BertAdam


In [ ]:
BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/'
convert_tf_checkpoint_to_pytorch.convert_tf_checkpoint_to_pytorch(
    BERT_MODEL_PATH + 'bert_model.ckpt',
BERT_MODEL_PATH + 'bert_config.json',
WORK_DIR + 'pytorch_model.bin')

shutil.copyfile(BERT_MODEL_PATH + 'bert_config.json', WORK_DIR + 'bert_config.json')


In [ ]:
from pytorch_pretrained_bert import BertConfig

bert_config = BertConfig('../input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/'+'bert_config.json')


In [ ]:
def convert_lines_onfly(example, max_seq_length,tokenizer,min_seq=0):
    max_seq_length -=2
    results =np.zeros((example.shape[0],max_seq_length+2),dtype=np.long)
    longest=min_seq
    for i,text in enumerate(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>longest:
            longest = len(tokens_a) 
        if len(tokens_a)>max_seq_length:
# if too long, randomly erase tokens
            a = np.arange(len(tokens_a))
            np.random.shuffle(a)
            tokens_a = list(np.array(tokens_a)[np.sort(a[:max_seq_length])])
# This is for head - tail instead of random
#            tokens_a=tokens_a[:max_seq_length//2]+tokens_a[len(tokens_a)-max_seq_length//2:]   
        results[i] = np.array(tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a)))
    results=results[:,:min(max_seq_length,longest)+2]
    return np.array(results)

In [ ]:
# We don't really need this.

def preprocess(data,hint=None,do_lower=True):
    '''
    Credit goes to https://www.kaggle.com/gpreda/jigsaw-fast-compact-solution
    '''
    punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~`" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'+"\n\t"
    def clean_special_chars(text, punct):
        for p in punct:
            text = text.replace(p, ' '+p+' ')
        return text
    
    def intonation(text,hint,do_lower):
        new_text=[]
        for token in text.split():
            if do_lower:
                new_text.append(token.lower())
            else:
                new_text.append(token)
            if hint and token.isupper() and len(token)>1:
                new_text.append(hint)
        return " ".join(new_text) 
    data = data.astype(str).apply(lambda x: clean_special_chars(intonation(x,hint,do_lower), punct))
    return data

In [ ]:
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)


In [ ]:
%%time


train_df = pd.read_csv(os.path.join(Data_dir,"train.csv")).sample(frac=partial_train,random_state=SEED)
print('loaded %d records' % len(train_df))
test_df=train_df.tail(100000)
train_df = train_df.head(((train_df.shape[0]-100000)//batch_size)*batch_size)
# Make sure all comment_text values are strings
sentences = sentences = preprocess(train_df['comment_text'].astype(str).fillna("DUMMY_VALUE")).values 
train_df=train_df.fillna(0)
# List all identities
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
y_columns=['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']+identity_columns
identity_sp = [ 'homosexual_gay_or_lesbian','muslim', 'black', 'white']
# Convert taget and identity columns to booleans

train_df = train_df.drop(['comment_text'],axis=1)

In [ ]:

def calculate_weights(train):
    has_identity = torch.sigmoid(10*(torch.tensor((train[identity_columns].fillna(0).max(axis=1)).values)-0.4))
    has_target = torch.sigmoid(10*(torch.tensor(train['target'].values)-0.4))
    weights = (torch.ones(train.shape[0],dtype=torch.float64)+has_identity+has_identity*(1-has_target)+has_target*(1-has_identity)) / 4
    weights = weights.to(dtype=torch.float32)
    return weights

In [ ]:
# make sure every batch has similar sentences length, and shuffle the batchs
sort_idx=np.argsort(np.array([len(x.split()) for x in sentences])).reshape(train_df.shape[0]//batch_size,batch_size)
np.random.shuffle(sort_idx)
sort_idx=sort_idx.reshape(train_df.shape[0])


In [ ]:
%%time
sentences = sentences[sort_idx]
X = sentences                #[train_df.idx]
y = train_df[y_columns].values[sort_idx]


In [ ]:
weights_tensor=calculate_weights(train_df)[sort_idx].repeat(len(y_columns),1).transpose(0,1)
weights_tensor[:,0]=weights_tensor[:,0]*(len(y_columns))/4
weights_tensor[:,6:]=weights_tensor[:,6:]*0.25


In [ ]:

train_dataset = torch.utils.data.TensorDataset(torch.arange(len(X)), torch.tensor((np.abs(2.0*y-1.0)**0.5*np.sign(y-0.5)+1)/2,dtype=torch.float), weights_tensor)


In [ ]:
input_model_file = "../working"
output_model_file = "bert_pytorch.bin"
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.cuda.empty_cache()

model = BertForSequenceClassification.from_pretrained("../working",cache_dir=None,num_labels=len(y_columns))

model.zero_grad()
_=model.to(device)

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
train = train_dataset

avg_val_loss = 0.
avg_val_accuracy = 0.
num_train_optimization_steps = int(num_epoch*len(train)/batch_size/accumulation_steps)

optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=lr,
                     warmup=0.05,
                     t_total=num_train_optimization_steps)
model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)
model=model.train()

tq = tqdm_notebook(range(num_epoch))
for epoch in tq:
#    optimizer.lr = epoch[0]
    torch.cuda.empty_cache()
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=False)

    avg_loss = 0.
    avg_accuracy = 0.
    lossf=None
    optimizer.zero_grad()
    tk0 = tqdm_notebook(train_loader,leave=False)
    for i,(ind_batch, y_batch, w_batch) in enumerate(tk0):
        ind_batch.requires_grad = False
        x_batch=torch.tensor(convert_lines_onfly(X[ind_batch.numpy()], MAX_SEQUENCE_LENGTH,tokenizer))
        y_pred = model(x_batch.to(device))
        loss =  F.binary_cross_entropy_with_logits(y_pred,y_batch.to(device),weight=w_batch.to(device))/accumulation_steps
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        #loss.backward()
        if (i+1) % accumulation_steps == 0:             # Wait for several backward steps
            optimizer.step()                            # Now we can do an optimizer step
            optimizer.zero_grad()

        if lossf:
            lossf = 0.98*lossf+0.02*loss.item()*accumulation_steps
        else:
            lossf = loss.item()
        tk0.set_postfix(loss = lossf)
        avg_loss += loss.item()*accumulation_steps / len(train_loader)
        avg_accuracy += torch.mean(((torch.sigmoid(y_pred[:,0])>0.5) == (y_batch[:,0]>0.5).to(device)).to(torch.float) ).item()/len(train_loader)
    tq.set_postfix(avg_val_loss=avg_val_loss,avg_val_accuracy=avg_val_accuracy,avg_loss=avg_loss,avg_accuracy=avg_accuracy)
    lossf=None


torch.save(model.state_dict(), output_model_file)

In [ ]:
# From baseline kernel

def calculate_overall_auc(df, model_name):
    true_labels = df[TOXICITY_COLUMN]>0.5
    predicted_labels = df[model_name]
    return metrics.roc_auc_score(true_labels, predicted_labels)

def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)

def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    bias_score = np.average([
        power_mean(bias_df[SUBGROUP_AUC], POWER),
        power_mean(bias_df[BPSN_AUC], POWER),
        power_mean(bias_df[BNSP_AUC], POWER)
    ])
    return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)



SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive

def compute_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan

def compute_subgroup_auc(df, subgroup, label, model_name):
    subgroup_examples = df[df[subgroup]>0.5]
    return compute_auc((subgroup_examples[label]>0.5), subgroup_examples[model_name])

def compute_bpsn_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df[(df[subgroup]>0.5) & (df[label]<=0.5)]
    non_subgroup_positive_examples = df[(df[subgroup]<=0.5) & (df[label]>0.5)]
    examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
    return compute_auc(examples[label]>0.5, examples[model_name])

def compute_bnsp_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df[(df[subgroup]>0.5) & (df[label]>0.5)]
    non_subgroup_negative_examples = df[(df[subgroup]<=0.5) & (df[label]<=0.5)]
    examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
    return compute_auc(examples[label]>0.5, examples[model_name])

def compute_bias_metrics_for_model(dataset,
                                   subgroups,
                                   model,
                                   label_col,
                                   include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model."""
    records = []
    for subgroup in subgroups:
        record = {
            'subgroup': subgroup,
            'subgroup_size': len(dataset[dataset[subgroup]>0.5])
        }
        record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
        record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
        record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
        records.append(record)
    return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)

In [ ]:
# Run validation
# The following 2 lines are not needed but show how to download the model for prediction
sentences = preprocess(test_df['comment_text'].astype(str).fillna("DUMMY_VALUE")).values
test_df=test_df.fillna(0)
sort_idx=np.flip(np.argsort(np.array([len(x.split()) for x in sentences])))
org_idx=np.argsort(sort_idx)
X = sentences[sort_idx]
test_preds = torch.zeros((len(X)))
x_test = torch.arange(len(X))
test = torch.utils.data.TensorDataset(x_test)
batchs = batch_size
mx = 320
model = BertForSequenceClassification(bert_config,num_labels=len(y_columns))
model.load_state_dict(torch.load(output_model_file ))
for p in model.parameters():
    p.requires_grad = False
_=model.to(device)
torch.cuda.empty_cache()
test_loader = torch.utils.data.DataLoader(test, batch_size=batchs, shuffle=False)
model=model.eval()
tk0 = tqdm_notebook(test_loader,leave=False)
tranct=0
for i,(ind_batch,) in enumerate(tk0):
    x_batch=torch.tensor(convert_lines_onfly(X[ind_batch.numpy()], mx,tokenizer))
    y_pred = model(x_batch.to(device))
    test_preds[i * batchs:(i+1) * batchs] = test_preds[i * batchs:(i+1) * batchs]+torch.sigmoid(y_pred[:, 0].cpu())
    tranct=tranct+batchs*(x_batch.shape[1]==mx)
    tk0.set_postfix(trunct=tranct,gpu_memory=torch.cuda.memory_allocated() // 1024 ** 2,batch_len=x_batch.shape[1])

    
MODEL_NAME = 'model1'
test_df[MODEL_NAME]=torch.sigmoid(torch.tensor(test_preds[org_idx])).numpy()
TOXICITY_COLUMN = 'target'
bias_metrics_df = compute_bias_metrics_for_model(test_df, identity_columns, MODEL_NAME, 'target')
bias_metrics_df
get_final_metric(bias_metrics_df, calculate_overall_auc(test_df, MODEL_NAME))